텐서보드로부터 값 뽑아내는 작업.

valid_AUROC 기준 최고성능인 epoch의 통계값들을 뽑아내면 된다.

In [1]:
from tensorboard.backend.event_processing import event_accumulator
import pandas as pd

In [2]:
# 사전 변수 넣기
tensorboard_dir = '/home/hschoi/leehyunwon/ECG-SNN/new_server/ver6/IF/tensorboard/SNN_MLP_ver6_poisson_2024-12-24-00-39-13_fold1'
tags = ['train_Loss','train_Accuracy','train_F1_micro','train_F1_weighted','train_AUROC_macro','train_AUROC_weighted','train_auprc', 'valid_Loss','valid_Accuracy','valid_F1_micro','valid_F1_weighted', 'valid_AUROC_macro','valid_AUROC_weighted','valid_auprc']
usable_tags = ['valid_Accuracy','valid_F1_micro', 'valid_AUROC_macro', 'valid_auprc']

In [3]:
# EventAccumulator를 사용하여 이벤트 파일 읽기
ea = event_accumulator.EventAccumulator(tensorboard_dir)
ea.Reload()

2024-12-27 16:25:31.148606: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-27 16:25:31.201075: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-27 16:25:31.975153: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
# 수집된 데이터를 저장할 딕셔너리
data = {}

# 각 태그의 데이터를 수집하여 DataFrame 생성
for tag in tags:
    if tag in ea.Tags()["scalars"]:  # 태그가 존재하는지 확인
        events = ea.Scalars(tag)
        data[tag] = {
            "epoch": [event.step for event in events],  # step을 epoch으로 사용
            "value": [event.value for event in events],
        }
    else:
        print(f"Tag '{tag}' not found in the TensorBoard logs.")

In [5]:
# 태그별 데이터프레임 생성 (속성 전환 정도도)
dataframes = {tag: pd.DataFrame(values) for tag, values in data.items()}

# 단일 데이터프레임으로 병합 (Epoch 기준)
merged_df = pd.DataFrame({"epoch": dataframes[tags[0]]["epoch"]}) # 모든 태그에 동일하게 존재하는 epoch만 따로 빼서 열 값으로 변경경
for tag in tags:
    if tag in dataframes:
        merged_df[tag] = dataframes[tag]["value"] # 각 태그의 value 값 이어붙이기

# 결과 확인
print(merged_df.head())

   epoch  train_Loss  train_Accuracy  train_F1_micro  train_F1_weighted  \
0      0    0.185447        0.887826        0.698549           0.698549   
1      1    0.122266        0.926403        0.798735           0.798735   
2      2    0.105129        0.935558        0.823197           0.823197   
3      3    0.095196        0.943056        0.842318           0.842318   
4      4    0.089674        0.946141        0.850316           0.850316   

   train_AUROC_macro  train_AUROC_weighted  train_auprc  valid_Loss  \
0           0.834035              0.834035     0.777752    0.135281   
1           0.894061              0.894061     0.872809    0.117877   
2           0.908709              0.908709     0.900859    0.108767   
3           0.917951              0.917951     0.915673    0.099419   
4           0.921813              0.921813     0.924157    0.095915   

   valid_Accuracy  valid_F1_micro  valid_F1_weighted  valid_AUROC_macro  \
0        0.920393        0.777103           0.7

하나의 텐서보드 로그에 대한 추출이 완료되었다. 이제 여기서 당장 필요한 녀석들만 추려보자.

In [6]:
# 사용할 태그와 존재하는 태그 교집합 계산
valid_tags = ['epoch'] + [tag for tag in usable_tags if tag in merged_df.columns]

# 교집합을 기반으로 필터링
filtered_df = merged_df[valid_tags]

print(filtered_df)

     epoch  valid_Accuracy  valid_AUROC_macro  valid_auprc
0        0        0.920393           0.879410     0.853131
1        1        0.933927           0.892645     0.889207
2        2        0.929473           0.906820     0.887333
3        3        0.942208           0.913553     0.911980
4        4        0.944720           0.914263     0.918531
..     ...             ...                ...          ...
295    295        0.981669           0.967822     0.981588
296    296        0.981726           0.967588     0.981324
297    297        0.980355           0.967298     0.980898
298    298        0.980527           0.965261     0.979185
299    299        0.978528           0.966866     0.980822

[300 rows x 4 columns]


In [7]:
# 가장 높은 valid_AUROC_macro 값을 갖는 행 추출
highest_auroc_row = filtered_df.loc[filtered_df['valid_AUROC_macro'].idxmax()]

# 결과 확인
print(highest_auroc_row)

epoch                278.000000
valid_Accuracy         0.978356
valid_AUROC_macro      0.968770
valid_auprc            0.981225
Name: 278, dtype: float64


In [8]:
# 저장(선택사항, 어차피 여기서 바꾸면 바로 나오는데..)
# filtered_df.to_csv("filtered_tensorboard_data.csv", index=False)